Installations

In [1]:

!pip install unidecode
!pip install contractions
!pip install nltk
!python -m spacy download en_core_web_sm
!pip install emoji
!pip install emoticons_list
!pip install emot
!pip install autocorrect
!pip install langdetect
!pip install textblob
!pip install krovetzstemmer


     |████████████████████████████████| 13.6 MB 3.6 MB/s eta 0:00:01


✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
ERROR: Could not find a version that satisfies the requirement emoticons_list
ERROR: No matching distribution found for emoticons_list


Imports

In [3]:
import os 
from tqdm.auto import tqdm 
from datetime import datetime
import pandas as pd
import re
from autocorrect import Speller
from spacy_langdetect import LanguageDetector
import spacy
import unidecode
from pathlib import Path
import itertools
import contractions
import nltk

from functools import reduce
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from nltk.tokenize import word_tokenize
import emoji
import emot 
from string import punctuation
from langdetect import detect_langs,detect
from textblob import TextBlob
from krovetzstemmer import Stemmer
from nltk.stem import WordNetLemmatizer

## Initialisations and general declarations

tqdm.pandas()
tokenizer = TweetTokenizer(preserve_case=False, 
                           strip_handles=True,
                           reduce_len=True)
nltk.download('stopwords')
nltk.download('punkt')
stopwords_english = stopwords.words('english')
nltk.download('wordnet')
stemmer = Stemmer()
lemmatizer = WordNetLemmatizer()

spell = Speller(lang = 'en')
emot_obj = emot.core.emot()



[nltk_data] Downloading package stopwords to
[nltk_data]     /home/azulcou/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/azulcou/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/azulcou/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


File Paths 

In [106]:
#file paths


slang_file_path = r"/mnt/c/Users/ksafa/OneDrive/Desktop/Omdena/omdena-colombia-socialnetwork/src/data/task-2-preprocessing/mapping/slang.txt"
folder_path = Path(r'/mnt/c/Users/ksafa/OneDrive/Desktop/Omdena/omdena-colombia-socialnetwork/src/data/task-1-data-scraping/Twitter')
output_folder_path = Path(r'/mnt/c/Users/ksafa/OneDrive/Desktop/Omdena/omdena-colombia-socialnetwork/src/data/task-2-preprocessing/preprocessed_raw_files/Twitter')

In [5]:

os.getcwd()

'/mnt/c/Users/ksafa/OneDrive/Desktop/Omdena/omdena-colombia-socialnetwork/src/tasks/task-2-preprocessing'

In [6]:
files_list = ['ColombiaSOS_ENGLISH_tweets_stweet.csv',
#               'ColombiaSOS_SPANISH_tweets_stweet.csv',
             'soscolombia_ENGLISH_tweets_stweet.csv']
              #'soscolombia_SPANISH_tweets_stweet.csv']

In [7]:
#Checking if columns are similar
for file in files_list:
    data = pd.read_csv(folder_path/file)
    print(data.columns)
    

Index(['created_at', 'id_str', 'conversation_id_str', 'full_text', 'lang',
       'favorited', 'retweeted', 'retweet_count', 'favorite_count',
       'reply_count', 'quote_count', 'quoted_status_id_str',
       'quoted_status_short_url', 'quoted_status_expand_url', 'user_id_str',
       'user_name', 'user_full_name', 'user_verified',
       'in_reply_to_status_id_str', 'in_reply_to_user_id_str', 'hashtags',
       'mentions', 'urls', 'media'],
      dtype='object')
Index(['created_at', 'id_str', 'conversation_id_str', 'full_text', 'lang',
       'favorited', 'retweeted', 'retweet_count', 'favorite_count',
       'reply_count', 'quote_count', 'quoted_status_id_str',
       'quoted_status_short_url', 'quoted_status_expand_url', 'user_id_str',
       'user_name', 'user_full_name', 'user_verified',
       'in_reply_to_status_id_str', 'in_reply_to_user_id_str', 'hashtags',
       'mentions', 'urls', 'media'],
      dtype='object')


In [8]:
#Slang dictionary 

f = open(slang_file_path, 'r')
slang_dict = {}
for line in f:
    k, v = line.strip().split('=')
    slang_dict[k.strip()] = v.strip()

f.close()

print(slang_dict)

{'AFAIK': 'As Far As I Know', 'AFK': 'Away From Keyboard', 'ASAP': 'As Soon As Possible', 'ATK': 'At The Keyboard', 'ATM': 'At The Moment', 'A3': 'Anytime, Anywhere, Anyplace', 'BAK': 'Back At Keyboard', 'BBL': 'Be Back Later', 'BBS': 'Be Back Soon', 'BFN': 'Bye For Now', 'B4N': 'Bye For Now', 'BRB': 'Be Right Back', 'BRT': 'Be Right There', 'BTW': 'By The Way', 'B4': 'Before', 'CU': 'See You', 'CUL8R': 'See You Later', 'CYA': 'See You', 'FAQ': 'Frequently Asked Questions', 'FC': 'Fingers Crossed', 'FWIW': "For What It's Worth", 'FYI': 'For Your Information', 'GAL': 'Get A Life', 'GG': 'Good Game', 'GN': 'Good Night', 'GMTA': 'Great Minds Think Alike', 'GR8': 'Great!', 'G9': 'Genius', 'IC': 'I See', 'ICQ': 'I Seek you (also a chat program)', 'ILU': 'ILU: I Love You', 'IMHO': 'In My Honest/Humble Opinion', 'IMO': 'In My Opinion', 'IOW': 'In Other Words', 'IRL': 'In Real Life', 'KISS': 'Keep It Simple, Stupid', 'LDR': 'Long Distance Relationship', 'LMAO': 'Laugh My A.. Off', 'LOL': 'Laug

In [9]:
data.head()

,created_at,id_str,conversation_id_str,full_text,lang,favorited,retweeted,retweet_count,favorite_count,reply_count,...,user_id_str,user_name,user_full_name,user_verified,in_reply_to_status_id_str,in_reply_to_user_id_str,hashtags,mentions,urls,media
0,2021-04-04T20:00:27+00:00,1378799647704166401,1378799647704166401,#SOSColombia @UEenColombia @llombartpatUE @Jo...,en,False,False,0,0,0,...,1710990733,PORPAZCONLACPI,#IglesiasConImpuestosYa,False,NaN,NaN,"#SOSColombia , #NarcoGoverment","UEenColombia , llombartpatUE , JoeBiden , Kama...",NaN,[]
1,2021-04-01T22:55:01+00:00,1377756415641714690,1377457680260956164,@ISAZULETA @alemEmbajador @CanadayColombia @Fr...,en,False,False,0,0,0,...,940438832,data1816,JOHANA🖤🇪🇨,False,1.377458e+18,139454733.0,#SOSColombia,"ISAZULETA , alemEmbajador , CanadayColombia , ...",NaN,[]
2,2021-03-31T02:39:20+00:00,1377088091286949888,1377088091286949888,"Hello, again!... @jguaido \r\n\r\n#30Mar #Sput...",en,False,False,9,0,0,...,1345129602,leonguerrero100,León Guerrero,False,NaN,NaN,"#30Mar , #SputnikVContraCovid , #SOSColombia",jguaido,NaN,[]
3,2021-04-24T21:50:10+00:00,1386075014508711942,1386075014508711942,@enoorman #SOSColombia @UEenColombia @llombar...,en,False,False,0,0,0,...,1710990733,PORPAZCONLACPI,#IglesiasConImpuestosYa,False,NaN,220670028.0,"#SOSColombia , #NarcoGoverment","enoorman , UEenColombia , llombartpatUE , JoeB...",NaN,[]
4,2021-04-23T14:44:42+00:00,1385605557655117825,1385605557655117825,Colombia needs helps form international organi...,en,False,False,5,18,1,...,311507486,AndresPerezHunt,Andres,False,NaN,NaN,"#SOSColombia , #ParoNacional28A",JoeBiden,NaN,[]


In [10]:
data.full_text[0]

'#SOSColombia @UEenColombia  @llombartpatUE @JoeBiden @KamalaHarris this #NarcoGoverment is killing us  @IntlCrimCourt NOBODY ACTS, MORE THAN ONE LÍDER MURDERED EVERY DAY @MisionONUCol @mbachelet NO ACTS IS ALSO CRIMINAL'

# Detecting Language 
 

In [11]:
#Using TextBlob

lang = TextBlob(data.full_text[0])
print(lang.detect_language())


en


In [12]:
#Using langdetect

print(detect_langs(data.full_text[0]))
print(detect(data.full_text[0]))

[en:0.9999973511093044]
en


In [13]:
# Needs some fix 
'''
#Using spacy

!python -m spacy download en_core_web_trf
!pip install spacy-transformers
nlp = spacy.load("en_core_web_trf")
nlp.add_pipe(LanguageDetector(), name='language_detector', last=True)
doc = nlp(text)
return doc._.language

#Using pycld2

!pip install pycld2
import pycld2 as cld2
_, _, _, detected_language = cld2.detect(data.full_text[0],  returnVectors=True)
print(detected_language)'   

''' 

'\n#Using spacy\n\n!python -m spacy download en_core_web_trf\n!pip install spacy-transformers\nnlp = spacy.load("en_core_web_trf")\nnlp.add_pipe(LanguageDetector(), name=\'language_detector\', last=True)\ndoc = nlp(text)\nreturn doc._.language\n\n#Using pycld2\n\n!pip install pycld2\nimport pycld2 as cld2\n_, _, _, detected_language = cld2.detect(data.full_text[0],  returnVectors=True)\nprint(detected_language)\'   \n\n'

Helper Functions

In [82]:
def language_detect(text):
    ##Using langdetect
    detect(text)
#     print('lang')

def lower_case_conversion(text):
#     print('lower')
    
    return text.lower()


def removing_html_tag(text):
#     print('html')
    clean_text = re.compile('<.*?>')
    return re.sub(clean_text,'',text)


def removing_urls(text):
#     print('url')
    return re.sub(r'https?://\S+|www\.\S+','',text)

def spelling_correction_en(text):
#     print('spell')
#     spell = Speller(lang = 'en')
    return spell(text)

def accented_to_ascii(text):
#     print('accented')
    return unidecode.unidecode(text)

def short_to_original(text):
#     print('short')
    #replace the slang word with meaning
    
    '''
    f = open(slang_file_path, 'r')
    slang_dict = {}
    for line in f:
        k, v = line.strip().split('=')
        slang_dict[k.strip()] = v.strip()

    f.close()
    '''
    text_tokens=text.split()
    for i,word in enumerate(text_tokens):
        if word in slang_dict.keys():
            text_tokens[i]=slang_dict[word]

    text_new=" ".join(text_tokens)

    return text_new

def standardizing(text):
#     print('standard')
    #One letter in a word should not be present more than twice in continuation
    return ''.join(''.join(s)[:2] for _, s in itertools.groupby(text))


def contractions_text(text):
#     print('contractions')
    expanded_words = []    
    for word in text.split():
      # using contractions.fix to expand the shortened words
      expanded_words.append(contractions.fix(word))   

    return ' '.join(expanded_words)

def stemming(text):
    return [stemmer.stem(token) for token in text.split()]
    

def lemmatization(text):
    return [lemmatizer.lemmatize(token) for token in text.split()]

def spanish(text):
    return " ".join(w for w  in nltk.wordpunct_tokenize(text) \
         if w.lower() not in words or not w.isalpha())
    
def extract_emojis(text):
#     print('extract emoji')
    return [''.join(c for c in text if c in emoji.UNICODE_EMOJI['en'])]

def extract_emoticons(text):
#     print('emoticons')
    
    return emot_obj.emoticons(text)['value']


def remove_punctuation(text):
#     print('punc')
    return text.translate(str.maketrans('', '', punctuation))

def remove_emojis(text):
#     print('emoji')
    return ''.join(c for c in text if c not in emoji.UNICODE_EMOJI['en'])

def remove_stopwords(text):
#     print('stop')
#     nltk.download('stopwords')
    stopwords_english = stopwords.words('english')
    return ' '.join([word for word in text.split() if word.lower() not in stopwords_english])

    
def remove_extra_spaces(text):
#     print('space')
    space_pattern = r'\s+'
    without_space = re.sub(pattern=space_pattern, repl=" ", string=text)
    return without_space

def remove_single_char(text):
#     print('single')
    single_char_pattern = r'\s+[a-zA-Z]\s+'
    without_sc = re.sub(pattern=single_char_pattern, repl=" ", string=text)
    return without_sc
    
def twitter_preprocessed_date(text):
    return datetime.strptime(text.split('T')[0], "%Y-%m-%d").date()

def remove_mention_text(text):
#     print('mention')
    mention_pattern = r'@\S+'
    return re.sub(pattern=mention_pattern, repl=' ', string=text)
    
def remove_digits(text):
#     print('digit')
    digits = r'[0-9]'
    return re.sub(pattern=digits, repl=' ', string=text)
    

In [83]:
    
def english_preprocessing(text, func_list):
    
    return reduce(lambda o, func: func(o), func_list, text)

In [93]:


preprocessed_data = data.copy()

## Storing emoticons and emojis separately

preprocessed_data['emoji_list'] = preprocessed_data['full_text'].apply(lambda x: extract_emojis(x))
                                                                       
preprocessed_data['emoticons_list'] = preprocessed_data['full_text'].apply(lambda x: extract_emoticons(x))

preprocessed_data['preprocessed_created_at'] = preprocessed_data['created_at'].apply(lambda x: twitter_preprocessed_date(x))


In [94]:
# To get Non English data separately
import nltk
nltk.download('words')
words = set(nltk.corpus.words.words())
spanish_func = [short_to_original, contractions_text, 
             standardizing, lower_case_conversion, 
             remove_emojis,remove_mention_text, removing_html_tag, 
             removing_urls, remove_punctuation, remove_extra_spaces,
             remove_digits, remove_single_char,lower_case_conversion, spanish]

preprocessed_data['full_text'][:2].progress_apply(lambda x:english_preprocessing(x, spanish_func))

[nltk_data] Downloading package words to /home/azulcou/nltk_data...
[nltk_data]   Package words is already up-to-date!


0    soscolombia narcogoverment acts líder murdered...
1                                          soscolombia
Name: full_text, dtype: object

In [90]:
##Stemming
english_func_list_stem  = [short_to_original, contractions_text, 
             standardizing, lower_case_conversion, 
             remove_emojis,remove_mention_text, removing_html_tag, 
             removing_urls, remove_punctuation, remove_extra_spaces,
             remove_digits, remove_single_char,lower_case_conversion, accented_to_ascii,
                      remove_stopwords,
                    stemming] 
preprocessed_data['full_text'][:2].progress_apply(lambda x: english_preprocessing(x, english_func_list))

0    [soscolombia, narcogoverment, killing, us, nob...
1                                  [soscolombia, help]
Name: full_text, dtype: object

In [89]:
## Lemmatization
english_func_list_lemma  = [short_to_original, contractions_text, 
             standardizing, lower_case_conversion, 
             remove_emojis,remove_mention_text, removing_html_tag, 
             removing_urls, remove_punctuation, remove_extra_spaces,
             remove_digits, remove_single_char,lower_case_conversion, accented_to_ascii,
                      remove_stopwords,
                    lemmatization] 
preprocessed_data['full_text'][:2].progress_apply(lambda x: english_preprocessing(x, english_func_list))

0    [soscolombia, narcogoverment, killing, us, nob...
1                                  [soscolombia, help]
Name: full_text, dtype: object

In [95]:


preprocessed_data['preprocessed_data'] = preprocessed_data['full_text'].progress_apply(lambda x: english_preprocessing(x, english_func_list_stem))

In [103]:
preprocessed_data

,created_at,id_str,conversation_id_str,full_text,lang,favorited,retweeted,retweet_count,favorite_count,reply_count,...,in_reply_to_status_id_str,in_reply_to_user_id_str,hashtags,mentions,urls,media,emoji_list,emoticons_list,preprocessed_created_at,preprocessed_data
0,2021-04-04T20:00:27+00:00,1378799647704166401,1378799647704166401,#SOSColombia @UEenColombia @llombartpatUE @Jo...,en,False,False,0,0,0,...,NaN,NaN,"#SOSColombia , #NarcoGoverment","UEenColombia , llombartpatUE , JoeBiden , Kama...",NaN,[],[],[],2021-04-04,"[soscolombia, narcogoverment, killing, us, nob..."
1,2021-04-01T22:55:01+00:00,1377756415641714690,1377457680260956164,@ISAZULETA @alemEmbajador @CanadayColombia @Fr...,en,False,False,0,0,0,...,1.377458e+18,139454733.0,#SOSColombia,"ISAZULETA , alemEmbajador , CanadayColombia , ...",NaN,[],[],[],2021-04-01,"[soscolombia, help]"
2,2021-03-31T02:39:20+00:00,1377088091286949888,1377088091286949888,"Hello, again!... @jguaido \r\n\r\n#30Mar #Sput...",en,False,False,9,0,0,...,NaN,NaN,"#30Mar , #SputnikVContraCovid , #SOSColombia",jguaido,NaN,[],[],[],2021-03-31,"[hello, mar, sputnikvcontracovid, soscolombia]"
3,2021-04-24T21:50:10+00:00,1386075014508711942,1386075014508711942,@enoorman #SOSColombia @UEenColombia @llombar...,en,False,False,0,0,0,...,NaN,220670028.0,"#SOSColombia , #NarcoGoverment","enoorman , UEenColombia , llombartpatUE , JoeB...",NaN,[],[],[],2021-04-24,"[soscolombia, narcogoverment, killing, us, nob..."
4,2021-04-23T14:44:42+00:00,1385605557655117825,1385605557655117825,Colombia needs helps form international organi...,en,False,False,5,18,1,...,NaN,NaN,"#SOSColombia , #ParoNacional28A",JoeBiden,NaN,[],[],[],2021-04-23,"[colombia, needs, help, form, international, o..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15272,2021-08-24T15:31:14+00:00,1430190973045383169,1430190973045383169,Former Paramilitary Criminals Reveal Colombian...,en,False,False,0,0,0,...,NaN,NaN,"#AUC , #ColombianArmy , #UniversidadDeNariño ,...",NaN,https://t.co/3JSeVAA7bo,[],[],[:/],2021-08-24,"[former, paramilitary, criminal, reveal, colom..."
15273,2021-08-24T04:24:56+00:00,1430023295940931586,1430023295940931586,23.8/21\r\n#SOSColombia\r\nThe extreme right a...,en,False,False,0,3,1,...,NaN,NaN,#SOSColombia,"USArmy , USArmyReserve , StateDept , USSupreme...",NaN,"[{""url"": ""https://pbs.twimg.com/ext_tw_video_t...",[],[:/],2021-08-24,"[soscolombia, extreme, right, police, assassin..."
15274,2021-08-24T02:26:09+00:00,1429993402284773376,1429993402284773376,"#SOSCOLOMBIA ⚠️In 2018 #EstebanMosquera, a stu...",en,False,False,18,22,3,...,NaN,NaN,"#SOSCOLOMBIA , #EstebanMosquera , #ESMAD , #Ne...",NaN,NaN,[],[⚠],[],2021-08-24,"[soscolombia, estebanmosquera, student, social..."
15275,2021-08-23T21:12:34+00:00,1429914485049991179,1429914485049991179,"Yes, it's still #SOSCuba like it's been for 62...",en,False,False,0,1,1,...,NaN,NaN,"#SOSCuba , #FreePalestine , #SOSColombia",NaN,NaN,[],[],[],2021-08-23,"[yes, still, soscuba, like, years, also, freep..."


In [6]:
files_list = ['ColombiaSOS_ENGLISH_tweets_stweet.csv',
#               'ColombiaSOS_SPANISH_tweets_stweet.csv',
             'soscolombia_ENGLISH_tweets_stweet.csv']
              #'soscolombia_SPANISH_tweets_stweet.csv']

In [107]:
#Checking if columns are similar
for file in files_list:
    data = pd.read_csv(folder_path/file)
    preprocessed_data = data.copy()

    ## Storing emoticons and emojis separately

    preprocessed_data['emoji_list'] = preprocessed_data['full_text'].apply(lambda x: extract_emojis(x))

    preprocessed_data['emoticons_list'] = preprocessed_data['full_text'].apply(lambda x: extract_emoticons(x))

    preprocessed_data['preprocessed_created_at'] = preprocessed_data['created_at'].apply(lambda x: twitter_preprocessed_date(x))

    preprocessed_data['preprocessed_data'] = preprocessed_data['full_text'].progress_apply(lambda x: english_preprocessing(x, english_func_list_stem))

    preprocessed_data.to_csv(output_folder_path/file)